# Operationalization of ALS on Azure AKS and ACI with Databricks

In this notebook we showcase how to train the recommendation algorithm ALS on Azure Databricks and operationalize it using Azure ML Service and Azure Kubernetes (AKS). 

This notebook is based on [this repository](https://github.com/dciborow/DB-Recs) by Daniel Ciborowski, dciborow@microsoft.com.

##### Setup
* Create a new Databricks cluster, using this configuration: `DB 4.1, Spark 2.3.0, Python3`.
* Attach the following PyPi libraries to the custer. [See here for help adding a library.](https://docs.databricks.com/user-guide/libraries.html).
    * Add Azure-cli via pypi: `azure-cli`.
    * Add AzureML via Pypi: `azureml-sdk[databricks]`.
    * Add pydocumentdb via Pypi: `pydocumentdb`.
* Attach CosmosDB uber jar to the library. The jar can downloaded at https://search.maven.org/artifact/com.microsoft.azure/azure-cosmosdb-spark_2.3.0_2.11/1.2.2/jar. Make sure you download the uber version.
* Add this repository utilies to the cluster as detailed in the [SETUP.md](../../SETUP.md##setup-guide-for-azure-databricks).


##### This notebook is broken down into four sections.
1. Service Creation
1. Training
1. Scoring
1. Operationalization

##### The following Azure services will be deployed into a new or existing resource group.
1. [Azure ML Service](https://docs.databricks.com/user-guide/libraries.html)
1. [Cosmos DB](https://azure.microsoft.com/en-us/services/cosmos-db/)
1. [Container Registery](https://docs.microsoft.com/en-us/azure/container-registry/)
1. [Container Instances](https://docs.microsoft.com/en-us/azure/container-instances/)
1. [Application Insights](https://azure.microsoft.com/en-us/services/monitor/)
1. [Storage Account](https://docs.microsoft.com/en-us/azure/storage/common/storage-account-overview)
1. [Key Vault](https://azure.microsoft.com/en-us/services/key-vault/)
1. [Azure Container Instances (ACI)](https://azure.microsoft.com/en-us/services/container-instances/)
1. [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/)

In [ ]:
import numpy as np
import os
import pandas as pd
import pprint
import shutil
import time, timeit
import urllib
import yaml
import json
import uuid
import matplotlib
import matplotlib.pyplot as plt

from azure.common.client_factory import get_client_from_cli_profile
from azure.mgmt.compute import ComputeManagementClient
import azure.mgmt.cosmosdb
import azureml.core
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.image import ContainerImage
from azureml.core.compute import AksCompute, ComputeTarget

import pydocumentdb
import pydocumentdb.document_client as document_client

import pyspark
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import Row

from reco_utils.dataset.cosmos_cli import find_collection, read_collection, read_database, find_database
from reco_utils.dataset.spark_splitters import spark_random_split

print("PySpark version:", pyspark.__version__)
print("Azure SDK version:", azureml.core.VERSION)

## I. Service Creation
Modify the following variables:

In [ ]:
# Select the services names
short_uuid = str(uuid.uuid4())[:4]
prefix = "reco" + short_uuid
data = "mvl"
algo = "als"

# Add your subscription ID
subscription_id = "XXXXXXXXXXXXXXXXXX"


In [ ]:
# Resource group and workspace
resource_group = prefix + "_" + data
workspace_name = prefix + "_"+data+"_aml"
workspace_region = "westus2"
print("Resource group:", resource_group)

# Columns
userCol = "UserId"
itemCol = "MovieId"
ratingCol = "Rating"
userColIndex = userCol.replace("Id","Index")
itemColIndex = itemCol.replace("Id","Index")

# CosmosDB
location = workspace_region
account_name = prefix + "-" + data + "-ds-sql"
DOCUMENTDB_DATABASE = "recommendations"
DOCUMENTDB_COLLECTION = "user_recommendations_" + algo

# AzureML
history_name = 'spark-ml-notebook'
model_name = data+"-"+algo+"-reco.mml" #NOTE: The name of a asset must be only letters or numerals, not contain spaces, and under 30 characters
service_name = data + "-" + algo
experiment_name = data + "_"+ algo +"_Experiment"

train_data_path = data + "Train"
test_data_path = data + "Test"

Import the Workspace class and check the azureml SDK version. The variable `exist_ok` checks if the workspace exists or not.

In [ ]:
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      exist_ok=True)

# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

In [ ]:
client = get_client_from_cli_profile(azure.mgmt.cosmosdb.CosmosDB)

async_cosmosdb_create = client.database_accounts.create_or_update(
    resource_group,
    account_name,
    {
        'location': location,
        'locations': [{
            'location_name': location
        }]
    }
)
account = async_cosmosdb_create.result()

my_keys = client.database_accounts.list_keys(
    resource_group,
    account_name
)

master_key = my_keys.primary_master_key
endpoint = "https://" + account_name + ".documents.azure.com:443/"

#db client
client = document_client.DocumentClient(endpoint, {'masterKey': master_key})

if find_database(client, DOCUMENTDB_DATABASE) == False:
    db = client.CreateDatabase({ 'id': DOCUMENTDB_DATABASE })
else:
    db = read_database(client, DOCUMENTDB_DATABASE)
# Create collection options
options = {
    'offerThroughput': 11000
}

# Create a collection
collection_definition = { 'id': DOCUMENTDB_COLLECTION, 'partitionKey': {'paths': ['/id'],'kind': 'Hash'} }
if find_collection(client,DOCUMENTDB_DATABASE,  DOCUMENTDB_COLLECTION) ==False:
    collection = client.CreateCollection(db['_self'], collection_definition, options)
else:
    collection = read_collection(client, DOCUMENTDB_DATABASE, DOCUMENTDB_COLLECTION)

In [ ]:
secrets = {
  "Endpoint": endpoint,
  "Masterkey": master_key,
  "Database": DOCUMENTDB_DATABASE,
  "Collection": DOCUMENTDB_COLLECTION,
  "Upsert": "true"
}
with open("secrets.json", "w") as file:
    json.dump(secrets, file)

## II. Training

For demo purposes, we are going to use the synthetic dataset [MovieRatings.csv](http://aka.ms/MovieRatings.csv). MovieRatings has around 227k ratings in the range of 0-10. 

In [ ]:
# Download Movie Lens
basedataurl = "http://aka.ms" 
datafile = "MovieRatings.csv"

datafile_dbfs = os.path.join("/dbfs", datafile)

if os.path.isfile(datafile_dbfs):
    print("found {} at {}".format(datafile, datafile_dbfs))
else:
    print("downloading {} to {}".format(datafile, datafile_dbfs))
    urllib.request.urlretrieve(os.path.join(basedataurl, datafile), datafile_dbfs)
    
data_all = sqlContext.read.format('csv')\
                     .options(header='true', delimiter=',', inferSchema='true', ignoreLeadingWhiteSpace='true', ignoreTrailingWhiteSpace='true')\
                     .load(datafile)    
data_all.printSchema()
display(data_all)

In [ ]:
train, test = spark_random_split(data_all, ratio=0.75, seed=123)

print("Train ({}, {})".format(train.cache().count(), len(train.columns)))
print("Test ({}, {})".format(test.cache().count(), len(test.columns)))

train_data_path_dbfs = os.path.join("/dbfs", train_data_path)
test_data_path_dbfs = os.path.join("/dbfs", test_data_path)

train.write.mode('overwrite').parquet(train_data_path)
test.write.mode('overwrite').parquet(test_data_path)
print("Train and test datasets saved to {} and {}".format(train_data_path_dbfs, test_data_path_dbfs))

One of the features of Azure ML is its [experimentation service](https://azure.microsoft.com/en-us/blog/experimentation-using-azure-machine-learning/). We can use it to log the metrics of the model using different hyperparamters. All this information can be viewed on the Azure dashboard. 

Let's start a training run by defining an experiment.

In [ ]:
myexperiment = Experiment(ws, experiment_name)
root_run = myexperiment.start_logging()

We will use ALS algorithm as the recommender model. For detailed explanation of ASL, please see our [Deep dive notebook on ALS](../02_modeling/als_deep_dive.ipynb).

In [ ]:
indexerContacts = StringIndexer(inputCol=userCol, outputCol=userColIndex, handleInvalid='keep').fit(data_all)
indexerRules = StringIndexer(inputCol=itemCol, outputCol=itemColIndex, handleInvalid='keep').fit(data_all)

als = ALS(maxIter=5, userCol=userColIndex, itemCol=itemColIndex, ratingCol=ratingCol, coldStartStrategy="drop")

# put together the pipeline
pipe = Pipeline(stages=[indexerContacts, indexerRules, als])

In [ ]:
# Regularization Rates
regs = [1, 0.1, 0.01, 0.001]
paramGrid = ParamGridBuilder().addGrid(als.regParam, regs).build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol=ratingCol, predictionCol="prediction")
cv = CrossValidator(estimator=pipe, evaluator=evaluator, estimatorParamMaps=paramGrid)
train.cache()
cvModel = cv.fit(train)


In [ ]:
# record a bunch of reg values in a ALS model
i = 0
for reg in regs:
    print(reg)
    # create a bunch of child runs
    with root_run.child_run("reg-" + str(reg)) as run:
        rmse = cvModel.avgMetrics[i]
        print("Root-mean-square error = " + str(rmse))
        
        # log reg, rmse and feature names in run history
        run.log("reg", reg)
        run.log("rmse", rmse)
        run.log_list("columns", train.columns)
        i += 1

In [ ]:
#Load all run metrics from run history into a dictionary object.
child_runs = {}
child_run_metrics = {}

for r in root_run.get_children():
    child_runs[r.id] = r
    child_run_metrics[r.id] = r.get_metrics()

#Now find the run with the lowest rmse
best_run_id = min(child_run_metrics, key = lambda k: child_run_metrics[k]['rmse'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', child_run_metrics[best_run_id])    
    

fig, ax = plt.subplots()

best_reg = child_run_metrics[best_run_id]['reg']
max_auc = child_run_metrics[best_run_id]['rmse']

reg_auc = np.array([(child_run_metrics[k]['reg'], child_run_metrics[k]['rmse']) for k in child_run_metrics.keys()])
reg_auc_sorted = reg_auc[reg_auc[:,0].argsort()]

ax.plot(reg_auc_sorted[:,0], reg_auc_sorted[:,1], 'r--')
ax.plot(reg_auc_sorted[:,0], reg_auc_sorted[:,1], 'bo')

ax.set_xlabel('reg', fontsize = 14)
ax.set_ylabel('rmse', fontsize = 14)
ax.set_title('rmse over reg', fontsize = 16)

# plot arrow
ax.arrow(x = best_reg + 0.45, y = max_auc, dx = -0.4, dy = 0, ls = '-', lw = 0.00001,
          width = 0.00001, head_width = 0.00002, head_length = 0.02)

# plot "best run" text
ax.text(x = best_reg, y = max_auc, s = 'Best Run', fontsize = 14)
#plt.show()

display(fig)

In [ ]:
# save model
cvModel.bestModel.write().overwrite().save(model_name)
print("Model saved")

In [ ]:
# upload the serialized model into run history record
model_dbfs = os.path.join("/dbfs", model_name)
mdl, ext = model_name.split(".")
model_zip = mdl + ".zip"
shutil.make_archive(mdl, 'zip', model_dbfs)
root_run.upload_file("outputs/" + model_name, model_zip)        

# now delete the serialized model from local folder since it is already uploaded to run history 
# shutil.rmtree(model_dbfs)
os.remove(model_zip)
        
# Declare run completed
root_run.complete()

##NOTE: service deployment always gets the model from the current working dir.
print("copy model from dbfs to local")
model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(model_name, model_local, True)

In [ ]:
pred = cvModel.transform(test)
display(pred)

re = RegressionEvaluator(metricName="rmse", labelCol=ratingCol,
                                predictionCol="prediction")
rmse = re.evaluate(pred)
print("Root-mean-square error = " + str(rmse))

## III. Scoring

In [ ]:
with open('secrets.json') as json_data:
    writeConfig = json.load(json_data)
    recs = cvModel.bestModel.stages[2].recommendForAllUsers(10)
    recs.withColumn("id",recs[userColIndex].cast("string")).select("id", "recommendations."+ itemColIndex)\
    .write.format("com.microsoft.azure.cosmosdb.spark").mode('overwrite').options(**writeConfig).save()

## IV. Operationalization

In [ ]:
#%%writefile score_sparkml.py

score_sparkml = """

import json
def init(local=False):
    global client, collection
    try:
      # Query them in SQL
      import pydocumentdb.document_client as document_client

      MASTER_KEY = '{key}'
      HOST = '{endpoint}'
      DATABASE_ID = "{database}"
      COLLECTION_ID = "{collection}"
      database_link = 'dbs/' + DATABASE_ID
      collection_link = database_link + '/colls/' + COLLECTION_ID
      
      client = document_client.DocumentClient(HOST, {'masterKey': MASTER_KEY})
      collection = client.ReadCollection(collection_link=collection_link)
    except Exception as e:
      collection = e
def run(input_json):      

    try:
      import json

      id = json.loads(json.loads(input_json)[0])['id']
      query = {'query': 'SELECT * FROM c WHERE c.id = "' + str(id) +'"' } #+ str(id)

      options = {}

      result_iterable = client.QueryDocuments(collection['_self'], query, options)
      result = list(result_iterable);
  
    except Exception as e:
        result = str(e)
    return json.dumps(str(result)) #json.dumps({{"result":result}})
"""


with open('secrets.json') as json_data:
    writeConfig = json.load(json_data)
    score_sparkml = score_sparkml.replace("{key}",writeConfig['Masterkey']).replace("{endpoint}",writeConfig['Endpoint']).replace("{database}",writeConfig['Database']).replace("{collection}",writeConfig['Collection'])

    exec(score_sparkml)

    with open("score_sparkml.py", "w") as file:
        file.write(score_sparkml)

In [ ]:
%%writefile myenv_sparkml.yml

name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - numpy==1.14.2
    - scikit-learn==0.19.1
    - pandas
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core
    - pydocumentdb

In [ ]:
#aci = azure container instance
aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Spark ML Databricks sample'}, 
    description = 'This is a great example.')

In [ ]:
mymodel = Model.register(model_path = model_name, # this points to a local file
                       model_name = model_name, # this is the name the model is registered as, am using same name for both path and name.                 
                       description = "ADB trained model",
                       workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

In [ ]:
models = [mymodel]
runtime = "spark-py"
conda_file = 'myenv_sparkml.yml'
driver_file = "score_sparkml.py"

# image creation
myimage_config = ContainerImage.image_configuration(execution_script = driver_file, 
                                    runtime = runtime, 
                                    conda_file = conda_file)


In [ ]:
# List images by ws
for i in ContainerImage.list(workspace = ws):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))

### IV.a. Azure Container Instances (ACI) deployment

In [ ]:
try:
    myservice = AciWebservice(ws, name=service_name)
except:
    myservice = Webservice.deploy_from_model(
        workspace=ws, 
        name=service_name,
        deployment_config = aci_config,
        models = models,
        image_config = myimage_config
    )

    myservice.wait_for_deployment(show_output=True)

print(yaml.dump(myservice.__dict__, default_flow_style=False))

In [ ]:
json2 = '["{\\"id\\":\\"5616\\"}"]'.encode()
res1_service = myservice.run(input_data = json2)
print(res1_service)

### IV.b. Azure Kubernetes Service (AKS) deployment

In [ ]:
image_id = ContainerImage.list(workspace = ws)[0].name
print(image_id)
image_version = ContainerImage.list(workspace = ws)[0].version
print(image_version)

In [ ]:
myimage = Image(workspace=ws, id="{}:{}".format(image_id, image_version)) 

#### Create AKS compute

It may take 20-25 minutes to create a new cluster

In [ ]:
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

aks_target.wait_for_completion(show_output = True)

print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

In [ ]:
scoring_url = aks_service.scoring_uri
service_key = aks_service.get_keys()[0]

In [ ]:
json2 = '["{\\"id\\":\\"5616\\"}"]'.encode()

In [ ]:
req = urllib.request.Request(scoring_url,data=json2)
req.add_header("Authorization","Bearer {}".format(service_key))
req.add_header("Content-Type","application/json")

tic = time.time()
with urllib.request.urlopen(req) as result:
    res = result.readlines()
    print(res)
    
toc = time.time()
t2 = toc - tic
print("Full run took %.2f seconds" % (toc - tic))